In [ ]:
from tensorflow.keras.layers import Input, Dense, Embedding, Conv1D, LeakyReLU, Flatten, Dropout
from tensorflow.keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
df = pd.read_csv('../data/clean_data.csv', index_col='Municipality')

# Get proper label
labels = df.filter(like='catalan')
label = df.filter(like='catalan').filter(like='total')

data = df.drop(list(labels.columns), axis=1)

In [ ]:
label[:] = pd.qcut(label['culture_knowledge_of_catalan_total'], q=4, labels=[0,1,2,3])
# label[label['culture_knowledge_of_catalan_total'] == 2] = 1
# label[label['culture_knowledge_of_catalan_total'] == 3] = 2

In [ ]:
label_data = pd.get_dummies(label['culture_knowledge_of_catalan_total'])
label_data_size = label_data.shape

In [ ]:
data = data.drop(['missing_count'], axis=1)
data['population_population_by_sex_main'] = pd.Categorical(data['population_population_by_sex_main'])
data['population_population_by_sex_main'] = data['population_population_by_sex_main'].cat.codes.astype(float)

data['economic_sectors_head_of_livestock_main'] = pd.Categorical(data['economic_sectors_head_of_livestock_main'])
data['economic_sectors_head_of_livestock_main'] = data['economic_sectors_head_of_livestock_main'].cat.codes.astype(float)

data['economic_sectors_cultivated_land_main'] = pd.Categorical(data['economic_sectors_cultivated_land_main'])
data['economic_sectors_cultivated_land_main'] = data['economic_sectors_cultivated_land_main'].cat.codes.astype(float)

data['culture_sports_facilities_main'] = pd.Categorical(data['culture_sports_facilities_main'])
data['culture_sports_facilities_main'] = data['culture_sports_facilities_main'].cat.codes.astype(float)

In [ ]:
def normalize(x):
    return (x - x.min()) / (x.max() - x.min())

In [ ]:
for col in data.columns:
    data[col] = normalize(data[col])

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(data, label_data, test_size=0.2,random_state=42)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2,random_state=42)

In [ ]:
PARAMS = {'LAYERS': [[1],
                     [10],
                     [100],
                     [100, 10]],
          'ALPHA': [0.1,
                    0.2,
                    0.3,
                    0.4],
          'DROPOUT': [0.1,
                      0.2,
                      0.3,
                      False],
          'BATCH_SIZE': [10,
                         20,
                         30]}

In [ ]:
DROPOUT = 0.1
ALPHA = 0.1
LAYERS = [100, 10]
BATCH_SIZE = 10
INPUT_SHAPE = data.shape[1]

def build_model(layers, dropout, alpha):
    inpt = Input(shape=INPUT_SHAPE)

    x = Flatten()(inpt)

    for layer in layers:
        x = Dense(layer, activation=LeakyReLU(alpha=alpha))(x)
        if dropout != False:
            x = Dropout(dropout)(x)


    out = Dense(4, activation="softmax")(x)

    model = Model(inpt, out)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=["accuracy"])

    return model

In [ ]:
m = build_model(LAYERS, DROPOUT, ALPHA)
m.summary()
m.fit(train_x, train_y, batch_size=BATCH_SIZE, validation_data=(val_x,val_y), epochs=100)

In [ ]:
mlp = MLPClassifier(alpha=0,
                           activation='logistic', max_iter=1000,
                           solver='adam',random_state=42)
mlp.fit(train_x,train_y)
y_pred = mlp.predict(val_x)

In [ ]:
def confusion_matrix(pred, true):
    pred = pd.Series(np.argmax(pred, axis=1))
    true = pd.Series(true.columns[np.where(true!=0)[1]])

    true.name = 'target'
    pred.name = 'predicted'
    cm = pd.crosstab(true.reset_index(drop=True), pred.reset_index(drop=True))
    cm = cm[cm.index]
    return cm

In [ ]:
confusion_matrix(y_pred, test_y)

In [ ]:
pred = pd.Series(np.argmax(m.predict(test_x), axis=1))
true = pd.Series(test_y.columns[np.where(test_y!=0)[1]])
f1_score(list(true.values), list(pred.values), average=None)